In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image
from typing import Tuple

# Root directory of the project
ROOT_DIR = os.path.abspath("/home/tracer/Рабочий стол/repos/Red_scull-ML-Hackaton/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR) # To find local version of the library

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
train_indexes: Tuple[int, int] = (0, 1400) #starting index, ending index
test_indexes: Tuple[int, int] = (1400, 1800)

class GarbageDataset(utils.Dataset):
    def load_dataset(self, images_dir: str, csv: pd.DataFrame, indexes: Tuple(int, int)):
        # Definig classes
        self._csv = csv
        self.add_source("dataset", 1, "garbage")
        
        # Defining images
        for i, image_name in enumerate(listdir(images_dir)):
            if i < indexes[0] or i > indexes[1]:
                continue
                
            image_path = os.path.join(images_dir, image_name)
            height, width = self._get_resolution(image_path)
            
            self.add_image('dataset', image_id=image_name, path=image_path, height=height, width=width)
            
    def _get_resolution(self, image_path: str):
        with Image.open(image_path) as img:
            width, height = img.size
        return (height, width)
        
    
    def _rle_to_mask(self, rle: str, height, width):
        rows, cols = height, width
        rle_numbers = [*map(int, rle.split())]
        rle_pairs = np.array(rle_numbers).reshape(-1,2)
        
        img = np.zeros(rows * cols, dtype=np.uint8)
        
        for index, length in rle_pairs:
            index -= 1
            img[index:index + length] = 255
        
        img = img.reshape(cols,rows)
        img = img.T
        
        return img.astype(np.bool)
        
    def load_mask(self, image_id):
        img_info = self.image_info[image_id]
        rle = self.csv[image_id]["rle"] # we should get rle value of row with image_name
        
        mask = self._rle_to_mask(rle, img_info['height'], img_info['width'])
        mask_count = self.csv[image_id]["count"]
        class_ids = np.fill(mask_count, self.class_names.index("garbage"))
        
        return mask, class_ids
    
    def image_referece(self, image_id):
        return elf.image_info[image_id]['path']
    

In [ ]:
csv = pd.read_csv('./train.csv')

train_set = GarbageDataset()
train_set.load_dataset('./images/', csv, train_indexes)
train_set.prepare()

print('Train: %d' % len(train_set.image_ids))

True